<a href="https://colab.research.google.com/github/ObjectMatrix/google-colab-notebook/blob/main/Underwriting_Actuaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai pandas numpy matplotlib seaborn sklearn

In [ ]:
import os
import re
import json

import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

with open('/content/drive/MyDrive/secrets.json', 'r') as f:
    secrets = json.load(f)

user_data = pd.read_csv('/content/drive/MyDrive/user_data.txt')

# Encode the gender column
user_data['gender'] = user_data['gender'].apply(lambda x: 1 if x == 'male' else 0)

print(user_data.head())
# Visualize the distribution of variables, correlations, and potential trends
sns.pairplot(user_data)
plt.show()

# Based on insights from the LLM and EDA, select important features
X = user_data[['age', 'gender', 'annual_mileage', 'vehicle_age', 'vehicle_value', 'accident_history']]
y = user_data['insurance_premium']

KEY = secrets['SECRET_KEY']
MODEL = secrets['MODEL']
openai.api_key = KEY

**Train the Model**

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)


**Evaluate the Model**

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
# Define a function to get LLM recommendations for underwriting
def get_recommendation(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response.choices[0].text.strip()


# Example: Get a recommendation for a specific customer profile
customer_profile = {"age": 45, "gender": "male", "annual_mileage": 12000}

recommendation = get_recommendation(f"Based on the customer profile {customer_profile}, should we offer them a lower, standard, or higher premium?")

print("Recommendation:", recommendation)


In [ ]:
prompt = """Assess the risk level of insuring a potential customer based on the following information:
- Age: 28
- Gender: Male
- Location: Los Angeles, California
- Driving history: One speeding ticket in the past three years, no accidents
- Vehicle: 2018 Honda Civic

Please provide a risk assessment and the factors contributing to this assessment."""


response = get_recommendation(prompt)
print(response)

The risk level of insuring this potential customer is considered to be low. The customer's age, gender, and location are all factors that contribute to this assessment. The customer's driving history is also taken into consideration, and the fact that they have only had one speeding ticket in the past three years is viewed as a positive. The vehicle that the customer is driving is also a factor, and the 2018 Honda Civic is a newer model that is less likely to be involved in an accident.
